# GDR3 Calibration Tool

### Contributors: René Andrae, Morgan Fouesneau

## Requirements

This package relies on input data and some astronomical tools

* `pandas` is necessary for reading the input data. Most functions assume `pandas.DataFrame` as inputs.
* `astropy` is used to compute Galactic coordinates when they are not provided. (some models use `cos(b)` as feature)
* `scikit-learn` (optional) when calibration models are based upon `BaseEstimator` (e.g, Extratrees, SVM)
* `joblib` (optional; requirement of scikit-learn) allows the package to load trained models from binary files
* `numpy` for usual array operations
* `git+https://github.com/scikit-learn-contrib/py-earth@v0.2dev` (optional) to support operations on MARS models (e.g., training, conversion)

## Preliminary verfications

In this section, we check the package installation and configuration.

In [1]:
# import sys
# uncommented, this line allows us to work with the 
# trunk version without installing the package.
# sys.path.insert(0, '../src/')

In [2]:
import gdr3apcal
print(""" 
Code version {0}
Package installed in: {1}
Model directory: {2}
""".format(gdr3apcal.__VERSION__, 
           gdr3apcal.config.__PACKAGE_DIR__, 
           gdr3apcal.config.modelsdir))

 
Code version 1.6
Package installed in: /home/codespace/.python/current/lib/python3.10/site-packages/gdr3apcal
Model directory: /home/codespace/.python/current/lib/python3.10/site-packages/gdr3apcal/models



## Converting a pyearth.Earth model to functions
In this section, we convert an `pyearth.Earth` model into python functions.

`py-Earth` is a Python implementation of _Jerome Friedman_'s Multivariate Adaptive Regression Splines (MARS) algorithm.
The main issue with this package is that it is often a pain to install. For instance, it is not compatible with `python 3.9` at the moment.
Therefore, we replace trained models by their functional forms.

We could eventually compile the resulting functions to gain some speed.

**references** 

[_friedman91]: Friedman, J. (1991). _Multivariate adaptive regression splines_. The annals of statistics, 19(1), 1–67. http://www.jstor.org/stable/10.2307/2241837

In [3]:
from glob import glob
try: 
    from gdr3apcal.mars_converter import convert_pyearth_models_from_dumps

    # converting files from the package model directory
    # This is not necessarily the place you have the models
    #
    # Note: you can directly convert models from the `pyearth.Earth` objects
    #       with `mars_converter.dump_pyearth_models`
    flist = glob(gdr3apcal.config.modelsdir + '/mars*joblib')

    # final source code exported into the following file
    output_python_file = 'mars_mh.py'

    # necessary description that is not stored with the model
    modelname = 'mh'
    features = ['teff_gspphot', 'logg_gspphot', 'mh_gspphot',
                'azero_gspphot', 'ebpminrp_gspphot', 'ag_gspphot',
                'mg_gspphot', 'cosb', 'libname_gspphot']
    label = 'mh_gspphot'
    groupby = 'libname_gspphot'    # multilib model applies on those groups.
    version = '0.7'                # to keep track

    # The following calls the conversion and prints the configuration of this model
    # One can copy-paste the output to the `configuration.yaml` and the source file
    # to the `modeldir`.
    if flist:  # run only if some models were found.
        fname, md5sum = convert_pyearth_models_from_dumps(flist, output_python_file)

        models = __import__(fname.replace('.py', '')).models
        models, md5sum

        config_ = {modelname: {'features': features,
                            'label': label,
                            'filename': fname, 
                            'md5sum': md5sum,
                            'groupby': groupby,
                            'callable': True,    # def model as functions
                            'version': version}}
        import yaml
        print(f'model {modelname:s} configuration \n')
        print(yaml.dump(config_))
except ImportError:
    # No pyearth
    pass

pyearth not installed


## Using fake data

In this section, we check that the code runs on fake data. The results are meaningless but this provides an operational test


In [4]:
import numpy as np
import pandas as pd

np.random.seed(1)

def generate_random_data(n_rows: int = 2) -> pd.DataFrame:
    """ Convenient data generator for tests """
    # Create pandas data frame with random data.
    columns = ['teff_gspphot', 'logg_gspphot', 'mh_gspphot', 'azero_gspphot', 
               'ebpminrp_gspphot', 'ag_gspphot', 'mg_gspphot', 'cosb']
    table = np.random.uniform(0.0, 1.0, [n_rows, len(columns)])
    lib = np.random.choice(['OB', 'A', 'MARCS', 'PHOENIX'], n_rows, p=[0.02, 0.08, 0.45, 0.45])
    df = pd.DataFrame(table, columns=columns)
    df['libname_gspphot'] = lib
    return df

df = generate_random_data(1000)
df

,teff_gspphot,logg_gspphot,mh_gspphot,azero_gspphot,ebpminrp_gspphot,ag_gspphot,mg_gspphot,cosb,libname_gspphot
0,0.417022,0.720324,0.000114,0.302333,0.146756,0.092339,0.186260,0.345561,MARCS
1,0.396767,0.538817,0.419195,0.685220,0.204452,0.878117,0.027388,0.670468,PHOENIX
2,0.417305,0.558690,0.140387,0.198101,0.800745,0.968262,0.313424,0.692323,PHOENIX
3,0.876389,0.894607,0.085044,0.039055,0.169830,0.878143,0.098347,0.421108,MARCS
4,0.957890,0.533165,0.691877,0.315516,0.686501,0.834626,0.018288,0.750144,MARCS
...,...,...,...,...,...,...,...,...,...
995,0.046043,0.791776,0.439389,0.704620,0.609383,0.914160,0.679539,0.204578,PHOENIX
996,0.989694,0.808360,0.051150,0.990246,0.084374,0.415975,0.888049,0.762113,PHOENIX
997,0.673047,0.533691,0.346976,0.899944,0.818957,0.945892,0.399558,0.233622,PHOENIX
998,0.723220,0.689099,0.543365,0.537107,0.809690,0.799585,0.847775,0.190362,PHOENIX


Checking code calls on random data. Results are obviously meaninless here.

In [5]:
from gdr3apcal import GaiaDR3_GSPPhot_cal
calib = GaiaDR3_GSPPhot_cal()
calib.calibrateMetallicity(df)

array([ -1.28753895,  -2.56961509,  -1.9815238 ,  -1.38497429,
        -2.11562404,  -3.86842766,          nan,  -5.972796  ,
        -1.59919205,  -0.95615446,  -1.1002689 ,  -3.08465728,
        -1.90344842,  -3.53318025,  -2.90827163,          nan,
        -8.7247589 ,  -4.7952413 ,  -0.43884928,  -1.07192129,
        -8.6614904 ,          nan,  -3.0767518 ,          nan,
                nan,  -2.72556544,  -5.48379592,  -1.65871099,
        -2.02180436,  -0.91815539,  -4.35305061,   0.042021  ,
        -1.47004977,  -3.14327755,  -6.85925004,  -1.70112828,
        -5.2289546 ,          nan,          nan,  -0.86644472,
                nan,  -2.19807296,  -0.69665872,  -1.76228086,
        -1.6393071 ,  -1.38838348,  -1.78356939,  -3.62308021,
        -4.6827955 ,  -3.80779127,  -3.00403125,  -2.82581413,
       -10.42330192,  -2.64961102, -10.82709814,   0.74919672,
        -1.59307375,  -1.56655713,  -3.76224393,  -3.46104206,
        -4.31788344,  -3.23467057,  -2.17726963,       

Check the behavior when missing features

In [6]:
try:
    calib.calibrateMetallicity(df.drop(columns=['azero_gspphot', 'ag_gspphot']))
except KeyError as e:
    print(e)

'Missing features from input data: azero_gspphot,ag_gspphot'


# Testing with demo data

In [7]:
from gdr3apcal.downloader import download_file

url = "https://keeper.mpdl.mpg.de/d/b518cfeaf1664dcb8312/files/?p=%2Fdemo-GSPSpec-vs-GSPPhot-metal.csv&dl=1"
fname = 'demo-data.csv'
source = download_file(url, fname)

In [8]:
import pandas as pd
import numpy as np
data = pd.read_csv(source)
data

,source_id,l,b,parallax,parallax_error,phot_g_mean_mag,phot_bp_mean_mag,phot_rp_mean_mag,teff_gspspec,logg_gspspec,...,flags_gspspec,teff_gspphot,logg_gspphot,mh_gspphot,azero_gspphot,ag_gspphot,ebpminrp_gspphot,distance_gspphot,mg_gspphot,libname_gspphot
0,5835725554428191104,326.634742,-3.749551,0.257299,0.015013,13.404951,14.487535,12.375557,4084.0,1.59,...,00000010000009999999999999999999999999999,5050.8380,2.5722,0.1408,2.6248,1.9191,1.0212,2400.7478,-0.4675,PHOENIX
1,5835725726226852480,326.666979,-3.789468,0.142775,0.014150,13.131085,14.374775,12.040055,4250.0,1.50,...,00000021000019999999999999999999999999999,4733.8633,2.0063,-0.2319,2.9756,2.1236,1.1413,3397.3354,-1.7435,PHOENIX
2,4658109465327385344,279.531278,-32.834735,2.373126,0.011887,11.898599,12.252738,11.380256,5712.0,3.68,...,00000010000009999999999999999999999999999,5780.7140,4.1391,0.3988,0.1639,0.1366,0.0724,415.3741,3.6750,MARCS
3,5835757577707659264,326.136689,-3.388114,0.216811,0.014882,12.342805,13.683682,11.215082,4007.0,1.09,...,00000000000009999999900999999999999999999,4587.9050,1.6570,-0.0356,3.2706,2.3009,1.2300,3731.9200,-2.8015,MARCS
4,5835725966745103232,326.658240,-3.739197,5.125078,0.017014,10.918342,11.265015,10.396759,6226.0,4.18,...,00000010000009999999900999901999999999999,6366.3037,4.3500,-0.3107,0.5201,0.4413,0.2381,196.9772,3.9724,PHOENIX
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,5917366526100736512,333.830817,-9.355939,0.910155,0.028003,11.400710,12.084267,10.601941,4557.0,1.99,...,00000000000009920019900009900999999999999,4731.8990,2.5729,0.1413,0.7525,0.5741,0.3000,1199.6442,0.4573,MARCS
96,5917121777358655872,332.231401,-8.348733,1.731636,0.012611,12.278474,12.560555,11.835876,6715.0,4.10,...,00000030000009999999999999999999999999999,6656.4570,4.1712,-0.4661,0.4192,0.3638,0.1968,560.2202,3.1889,A
97,5917201015234969472,333.046934,-9.722948,4.737724,0.016760,9.868277,10.159750,9.405288,6439.0,4.14,...,00000000000001299000000999900029999999999,6275.1190,4.0375,-0.6100,0.2787,0.2393,0.1297,209.5456,3.0274,A
98,5917366869698143360,333.830908,-9.314400,0.408543,0.021761,11.831022,12.569979,10.996258,4389.0,1.52,...,00000000000000012999900009900999999999909,4277.1460,1.3717,-0.5854,0.4209,0.3133,0.1652,4001.2747,-1.4408,MARCS


In [9]:
from gdr3apcal import GaiaDR3_GSPPhot_cal
# calib = GaiaDR3_GSPPhot_cal()     # reusing the previous object for memory
calib

Calibration Models
    mh

In [10]:
teff_raw_r = np.sqrt(np.mean((data['teff_gspphot'] - data['teff_gspspec']) ** 2))
mh_raw_r = np.sqrt(np.mean((data['mh_gspphot'] - data['mh_gspspec']) ** 2))
print(f' Teff raw RMS difference: {teff_raw_r:3.4f} K')
print(f'[M/H] raw RMS difference: {mh_raw_r:3.4f} dex')

 Teff raw RMS difference: 669.3194 K
[M/H] raw RMS difference: 0.7394 dex


In [11]:
import warnings
with warnings.catch_warnings():
    # Catching the sklearn version warning
    warnings.simplefilter('ignore')
    try:
        cal_teff = calib.calibrateTeff(data)
    except AttributeError:
        print('no teff calibration model')
        cal_teff = data['teff_gspphot']
    try:
        cal_mh = calib.calibrateMetallicity(data)
    except AttributeError:
        print('no [M/H] calibration model')
        cal_mh = data['mh_gspphot']

teff_cal_r = np.sqrt(np.mean((cal_teff - data['teff_gspspec']) ** 2))
mh_cal_r = np.sqrt(np.mean((cal_mh - data['mh_gspspec']) ** 2))
print(f' Teff calibrated RMS difference: {teff_cal_r:3.4f} K (previously {teff_raw_r:3.4f})')
print(f'[M/H] calibrated RMS difference: {mh_cal_r:3.4f} dex (previously {mh_raw_r:3.4f})')

no teff calibration model
Automatically adding "cos(b)" from "b" [assuming degrees].
Automatically adding "cos(b)" from "b" [assuming degrees].
Automatically adding "cos(b)" from "b" [assuming degrees].
 Teff calibrated RMS difference: 669.3194 K (previously 669.3194)
[M/H] calibrated RMS difference: 0.6662 dex (previously 0.7394)
